# 3 - Importance of Dragons and Barons

[In the last project](https://github.com/inm2/Damwon-Analysis/blob/main/2%20-%20Factors%20of%20Dominance.ipynb), I showed how important of a factor gold is towards the performance of a team in League of Legends. In this project, I will look at how dragons independently factor into wins. [Dragons allow for different types of stat boosts in the game and influence the flow of the game.](https://leagueoflegends.fandom.com/wiki/Cloud_Drake) The first dragon spawns at the 5 minute mark and has a respawn timer of 5 minutes. 

To keep with the DAMWON Gaming theme, I used the following LCK Game Data:  
- Spring Split (Regular Season and Playoffs): 238 Games
- Summer Split (Regular Season and Playoffs): 225 Games
- Regional Finals: 10 Games
- Total: 473 Games

For the initial process, I imported the primary data set aggregated and located on Oracle's Elixr. Since I had issues with importing team, dragons, and baron data on MySQL, I filtered through the data after the dataframe was created.

In this project I looked at the following impacts in relation to match results:  
- Taking the first dragon   
- Dragon differential (DD)  
- Barons  

In [10]:
#Import Dataset
import pandas as pd
import numpy as np
#I experienced false positives when creating new columns, so I added this line to remove the warnings
pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_csv(r'2020LeagueInfo.csv')

In [3]:
#Filter by league and by position
is_LCK = df.league == "LCK"
is_team = df.position == "team"
LCKteam_filter = is_LCK & is_team
league = df[LCKteam_filter]

## Taking the First Dragon

In 2019, [Tim Sevenhuysen's Dragon Analysis](https://oracleselixir.com/blog/post/10689/early-game-rating-2-0) shows the importance of killing a dragon in the early stage of the game (~15 minutes). His analysis shows a difference of 1 dragon results in a 40-60% Win Rate. In the next two sections, I want to analyze, based off of the LCK data, how much of an impact taking the first dragon in the game has on its outcome, as well as do a reverification of the dragon differential on the 2020 LCK dataset.

The value of dragons differ greatly, and I think that deserves it's own analysis on specific impact per time range. I don't have that metadata, so I am utilizing every dragon into one category.

Based on the input data, the more challenging aspect was to figure out the logic in which to compare one metric to another, i.e. how to compare information from two teams?  

In the source data, the information defines a team, or playerid, in a match as playerid = 100 (Team 1) or playerid = 200 (Team 2). The following code shows how many times Team 1, or playerid = 100, has won a match.

The parameter result is binary and a 1 indicates a win, while 0 is a loss. 

Since the dataset has two entries, to account for both team's stats, per game, there is double the total amount of games (473 games * 2 = 946 rows). This value is represented as a null, or 0 = 473, to account for the other team's status.

In [18]:
#Teams are defined by playerid = 100 or 200, where 100 = Team 1 and 200 = Team 2
conditions = [(league['playerid'] == 100) & (league['result'] == 1),
              (league['playerid'] == 100) & (league['result'] == 0)]

#Defines the conditions above 
values = ['Team1_win','Team1_lose']

#Goes through the dataset to verify if the conditions are true or false
#Creates a new column in the dataset
league['Team Status'] = np.select(conditions, values)

#Gets a count of the amount of times each condition appears in the created column 
print("How many times has Team 1 won or lost?")
print(league.groupby(['Team Status']).size())

How many times has Team 1 won or lost?
Team Status
0             473
Team1_lose    211
Team1_win     262
dtype: int64


From the results above, Team 1 won 55% of the time. 

After this preliminary step, I wanted to go further and see how many times a team in general wins the game if they take the first dragon. 

The variable, 'firstdragon,' is also binary, where a 1 indicates that the team has taken the first dragon of the game.

In [19]:
#What % of teams win if they get the first dragon
conditions = [(league['firstdragon'] == 1) & (league['result'] == 1),
              (league['firstdragon'] == 1) & (league['result'] == 0)]

values = ['1st Dragon Win', '1st Dragon Loss']

league['1st Dragon'] = np.select(conditions, values)
print("\nHow many times does a team win if they get the first dragon?")
print(league.groupby(['1st Dragon']).size())


How many times does a team win if they get the first dragon?
1st Dragon
0                  473
1st Dragon Loss    202
1st Dragon Win     271
dtype: int64


From the results above, a team that takes the first dragon wins 57.29% of the time. 

## Dragon Differential

Dragon Differential (DD) is the difference between the amount of dragons Team 2 has slain vs the Team 1's amount. The purpose of this section is to show how likely a team is to win if they have a higher count of dragon's than their opponents. 

The code .diff() takes the amount of dragons Team 1 has and substracts it into the amount of dragons awarded to Team 2. 

Unlike the previous section, Team 2 will be the primary playerid and their result of the match (Win or Loss) will be taken. I divided the DD into three main categories, a positive DD (+Dragon), a neutral DD (Neutral), and a negative DD (-Dragon).

Thus the logic can be broken down into the following condition statements:

If Team 2 has a Win and a -DD (T2 Win (-Dragon)):  
```(league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] < 0)```  

If Team 2 has a Win and a +DD (T2 Win (+Dragon)):  
```(league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] > 0)```

If Team 2 has a Win and a neutral DD (T2 Win (Neutral)):  
```(league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] == 0)```  

If Team 2 has a Loss and a -DD (T1 Win (+Dragon)):  
```(league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] < 0)``` 

If Team 2 has a Loss and a +DD (T1 Win (-Dragon)):  
```(league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] > 0)```  

If Team 2 has a Loss and a neutral DD (T1 Win (Neutral)):  
```(league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] == 0)```  

In [15]:
#shift team 1 dragon rows to match team 2
league['drag_diff'] = league['dragons'].diff()

#Ignore Half the dataset
#Team 2 Wins or Loses and Has a +,-,or neutral dragon difference
conditions = [(league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] < 0),
              (league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] > 0),
              (league['playerid'] == 200) & (league['result'] == 1) & (league['drag_diff'] == 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] < 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] > 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['drag_diff'] == 0)]

values = ['T2 Win (-Dragon)','T2 Win (+Dragon)', 'T2 Win (Neutral)', 'T1 Win (+Dragon)', 'T1 Win (-Dragon)', 'T1 Win (Neutral)']

league['WinDrag'] = np.select(conditions, values)
print("Grouping the times Team 1 or 2 Won Based on Dragon Difference?")
print(league.groupby(['WinDrag']).size())

Grouping the times Team 1 or 2 Won Based on Dragon Difference?
WinDrag
0                   473
T1 Win (+Dragon)    191
T1 Win (-Dragon)     38
T1 Win (Neutral)     33
T2 Win (+Dragon)    165
T2 Win (-Dragon)     27
T2 Win (Neutral)     19
dtype: int64


Combining the T1 Win (+Dragon) and T2 Win (+Dragon) categories (356 wins), in the LCK, having a positive dragon differential resulted in a win % of 75.26, indicating the importance of dragons alone. For future analysis, it might be worth looking into difference increments of DD to see the impact at that level.

## Barons

Baron Nashoor is the most powerful monster in League of Legends. Taking out the monster nets a [large buff](https://leagueoflegends.fandom.com/wiki/Baron_Nashor_(League_of_Legends)) towards attack damage, ally minion damage, and more. Slaying a baron is important towards setting a team up for success in a game. 

Similar to Dragon Difference, Baron Difference, takes the barons Team 2 has and subtracts it by Team 1's. The logic is the same as the Dragon Difference. To show how the notation can be represented in the case of only Team 2's perspective, here are the following cases:

Team 2 Wins with Negative Barons (T2 Win (-Baron)):    
```(league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] < 0)```  

Team 2 Wins with Positive Barons (T2 Win (+Baron)):    
```(league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] > 0)```

Team 2 Wins with Neutral Barons (T2 Win (Neutral)):  
```(league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] == 0)```

Team 2 Loses with Negative Barons a.k.a. Team 1 Wins with Positive Barons (T2 Loss (-Baron)):   
```(league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] < 0)```

Team 2 Loses with Positive Barons a.k.a. Team 1 Wins with Negative Batons (T2 Loss (+Baron)):    
```(league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] > 0)```

Team 2 Loses with Neutral Barons a.k.a. Team 1 Wins with Neutral Barons (T2 Loss (Neutral)):    
```(league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] == 0)```

In [16]:
#Baron Analysis
league['baron_diff'] = league['barons'].diff()
conditions = [(league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] < 0),
              (league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] > 0),
              (league['playerid'] == 200) & (league['result'] == 1) & (league['baron_diff'] == 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] < 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] > 0),
              (league['playerid'] == 200) & (league['result'] == 0) & (league['baron_diff'] == 0)]

values = ['T2 Win (-Baron)','T2 Win (+Baron)', 'T2 Win (Neutral)', 'T2 Loss (-Baron)', 'T2 Loss (+Baron)', 'T2 Loss (Neutral)']

league['WinBaron'] = np.select(conditions, values)
print("Grouping the times Team 1 or 2 Won Based on Baron Difference?")
print(league.groupby(['WinBaron']).size())

Grouping the times Team 1 or 2 Won Based on Baron Difference?
WinBaron
0                    473
T2 Loss (+Baron)      21
T2 Loss (-Baron)     181
T2 Loss (Neutral)     60
T2 Win (+Baron)      167
T2 Win (-Baron)       14
T2 Win (Neutral)      30
dtype: int64


Combining the wins of Team 1 (T2 Loss (-Baron)) and Team 2 (T2 Win (+Baron)), which equals to 348, in the LCK, having a positive amount of Barons results in winning 73.57% of all games in the LCK, indicating barons are independently significant.

Throughout this project and the previous one, I looked at the most important factors towards winning a game of League of Legends, Gold Difference, Dragons, and Barons. Now that I've looked at these individually, the final step is to do an analysis to determine if wins and losses can be predicted given this information. 